In [15]:
!pip install transformers torch
!pip install -U transformers[torch] accelerate



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
pip show accelerate


Name: accelerate
Version: 1.6.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: c:\Users\Aryan Deshpande\AppData\Local\Programs\Python\Python312\Lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [20]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# Load pre-trained tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set to evaluation mode
model.eval()

# Input text
input_text = "The sun is shining and the sky is"

# Encode input
inputs = tokenizer.encode(input_text, return_tensors='pt')

# Generate predictions
with torch.no_grad():
    outputs = model.generate(inputs, max_length=inputs.shape[1] + 1, do_sample=False)

# Decode output
predicted_text = tokenizer.decode(outputs[0])

print("Input:", input_text)
print("Prediction:", predicted_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: The sun is shining and the sky is
Prediction: The sun is shining and the sky is clear


In [21]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Prepare dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=r"D:\RDR2_FullTranscript.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train the model
trainer.train()


c:\Users\Aryan Deshpande\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,2.727200
1000,2.583000
1500,2.550600
2000,2.475500


TrainOutput(global_step=2339, training_loss=2.567505013285258, metrics={'train_runtime': 1968.2135, 'train_samples_per_second': 1.188, 'train_steps_per_second': 1.188, 'total_flos': 152790515712000.0, 'train_loss': 2.567505013285258, 'epoch': 1.0})

In [22]:
outputs = model.generate(
    inputs,
    max_length=inputs.shape[1] + 5,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

for i, output in enumerate(outputs):
    print(f"Prediction {i+1}: {tokenizer.decode(output)}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prediction 1: The sun is shining and the sky is bright.

Un
Prediction 2: The sun is shining and the sky is filled with light… we
Prediction 3: The sun is shining and the sky is clear. A strong wind


In [31]:
test_inputs = [
    "I've Known Good White People and",
    "Far As Your Concerned, I Am The",
    "All Them Years, Dutch, For",
    "Some Jobs Aren’t For Saving, And Some Legacies Are For",
    "Vengeance Is An"
]

for text in test_inputs:
    inputs = tokenizer.encode(text, return_tensors='pt')
    outputs = model.generate(
        inputs,
        max_length=inputs.shape[1] + 10,  # allow up to 10 tokens total
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.9
    )
    predicted_text = tokenizer.decode(outputs[0])
    print(f"\nInput: {text}")
    print(f"Prediction: {predicted_text}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Input: I've Known Good White People and
Prediction: I've Known Good White People and C’mon. We ain’t


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Input: Far As Your Concerned, I Am The
Prediction: Far As Your Concerned, I Am The One

Micah Bell: You can tell


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Input: All Them Years, Dutch, For
Prediction: All Them Years, Dutch, For My Own, I Don’t Believe In


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Input: Some Jobs Aren’t For Saving, And Some Legacies Are For
Prediction: Some Jobs Aren’t For Saving, And Some Legacies Are For Sells

Arthur Morgan: So many that

Input: Vengeance Is An
Prediction: Vengeance Is An Abomination
[They drive off the cliff.
